In [34]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import *
import math
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pyfinance.ols import PandasRollingOLS
import re
from dateutil.relativedelta import *

import warnings
warnings.filterwarnings("ignore")

In [35]:
#读取全品种历史数据
df = pd.read_excel('.//B.1 - Choice数据//全品种期货主连数据.xlsx')

#筛选出目标品种
PZ_list = pd.read_excel('.//B.1 - Choice数据//future-factor_Choice版.xlsx')
PZ_list =  PZ_list['品种'].values.tolist()
df = df[df['品种'].isin(PZ_list)]
df

,时间,代码,对应主力,收盘价,前收盘价,收盘涨跌幅,结算价,前结算价,结算涨跌幅,成交量,成交金额,持仓量,持仓量变化,品种,板块
0,2013-12-31,A0.DCE,A1405.DCE,4441.0,4411.0,0.006801,4434.0,4412.0,0.004986,60974.0,2.704132e+09,66464.0,-4306.0,A,农产品
1,2014-01-02,A0.DCE,A1409.DCE,4322.0,4321.0,0.000231,4311.0,4312.0,-0.000232,27710.0,1.194784e+09,79750.0,13286.0,A,农产品
2,2014-01-03,A0.DCE,A1409.DCE,4326.0,4322.0,0.000925,4316.0,4311.0,0.001160,18792.0,8.111754e+08,77098.0,-2652.0,A,农产品
3,2014-01-06,A0.DCE,A1409.DCE,4334.0,4326.0,0.001849,4331.0,4316.0,0.003475,22442.0,9.721491e+08,75212.0,-1886.0,A,农产品
4,2014-01-07,A0.DCE,A1409.DCE,4338.0,4334.0,0.000923,4336.0,4331.0,0.001154,23942.0,1.038202e+09,68690.0,-6522.0,A,农产品
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141435,2023-01-19,scm.INE,sc2303.INE,549.3,563.5,-0.025200,556.7,558.8,-0.003758,106186.0,5.912025e+10,25953.0,-4303.0,sc,能源化工
141436,2023-01-20,scm.INE,sc2303.INE,563.9,549.3,0.026579,558.9,556.7,0.003952,94337.0,5.272733e+10,23213.0,-2740.0,sc,能源化工
141437,2023-01-30,scm.INE,sc2303.INE,552.4,563.9,-0.020394,555.0,558.9,-0.006978,29861.0,1.657563e+10,22816.0,-397.0,sc,能源化工
141438,2023-01-31,scm.INE,sc2303.INE,540.5,552.4,-0.021542,547.4,555.0,-0.013694,99683.0,5.457039e+10,22771.0,-45.0,sc,能源化工


In [36]:
#剔除上市半年内（125个交易日）的合约
for c in PZ_list:
    if df.loc[df['品种']==c]['时间'].astype(str).values[0] != '2013-12-31':
        df = df.drop(df.loc[df['品种']==c].index[:125])
    else:
        pass
    
df = df.reset_index(drop=True)

In [37]:
#计算平均成交量，按需剔除流动性弱的品种
df['平均成交'] = 0
for i in PZ_list:
    ave_vol = df.loc[df['品种']==i]['成交量'].shift().rolling(20).mean()
    df.loc[(df['品种']==i), '平均成交'] = ave_vol

In [38]:
#用于计算持仓金额
multi = pd.read_excel('./B.1 - Choice数据/主连数据_choice.xlsx',sheet_name='合约乘数')
multi = multi.iloc[2:,np.r_[0,2]]
multi.columns = ['主连品种','合约乘数']
multi = multi.dropna()
multi = multi.reset_index(drop=True)
multi['品种'] = 0
multi['交易所'] = 'INE'
for i in range(len(multi)):
    multi['品种'][i] = re.findall('[a-zA-Z]+',multi['主连品种'][i])[0]
    multi['交易所'][i] = multi.loc[i,'主连品种'][-3:]
multi.loc[multi['交易所']=='INE','品种'] = multi.loc[multi['交易所']=='INE','品种'].apply(lambda x:x[:2])

In [6]:
def multisum(x):   # 定义连加函数,返回连加后的最后一个值
    
    summ = x.cumsum()
    summ = summ.iloc[-1]
    
    return summ    

def multiprod(x):   # 定义连乘函数，返回连乘后的最后一个值
    
    prod = x.cumprod()
    prod = prod.iloc[-1]
    
    return prod

In [7]:
def LongShort(table,ret,position):
    """计算Long-Short组合收益率
    
    Parameters:
     table -  已包含多空方向的dataframe  
     ret - 收盘涨跌幅 or 结算涨跌幅
     position - ts_position or cs_position 指定时序动量还是截面多空类因子
    
    Returns：
     包含多头组合retn，空头组合ret和多空组合ret的dataframe

    """
   
    #多头组合
    table_long = table.loc[table[position] == 1]  
    ret_long = pd.DataFrame(table_long.groupby(by='时间')[ret].mean()) 

    #空头组合
    table_short = table.loc[table[position] == -1]
    ret_short = pd.DataFrame(table_short.groupby(by='时间')[ret].mean())

    #时序动量因子
    long_short = pd.merge(ret_long,ret_short,how='outer',on='时间').fillna(0).sort_index()
    long_short.columns = ['long_return','short_return']
    long_short['factor_return'] = long_short['long_return'] - long_short['short_return']

    return long_short

In [8]:
def cs_position(table):
    """标记截面类策略多、空头
    
    Parameters:
     table - 已包含Rank的dataframe
    
    Returns:
     包含多空方向的dataframe
     
    """
    
    time = set(table['时间'])
    time = list(time)
    time.sort()
    
    #截面动量仓位
    table['cs_position'] = 0
    for day in time:
        n = len(table.loc[(table['时间']==day), :])
        select = n * 0.2
        if (select > 0.2) & (select<1):
            select = 1
        else:
            select = round(n * 0.2)
        table.loc[(table['时间']==day) & (table['rank']<= select), 'cs_position'] = 1
        table.loc[(table['时间']==day) & (table['rank']> n-select), 'cs_position'] = -1
        
    return table    

In [9]:
class TimeSeries_ave(object):  
    """时序动量，所有品种收益直接平均
    
    Parameters:
     R - retroactive period
     L - lag period
     ret - 收盘涨跌幅 or 结算涨跌幅
     limit - 成交量阈值，剔除低于该值的品种
    
    """

    def __init__(self,R,L,ret,limit):

        self.R = R
        self.L = L
        self.ret = ret
        self.limit = limit
        
    def ts_mom(self):      #时序动量
        
        table = df.copy()
        
        table['ts_mom'] = 0
        for j in PZ_list:
            table.loc[(table['品种']==j), 'ts_mom'] = table.loc[table['品种']==j][self.ret].shift(self.L+1).rolling(self.R).apply(multisum)
        table = table.dropna()
        
        table['ts_position'] = np.sign(table['ts_mom'])
        
        #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #直接平均
        table['factor_return'] = table[ret]*table['ts_position']
        
        return table

In [10]:
class TimeSeries_ls(object): 
    """时序动量，品种收益多头减空头
    
    Parameters:
     R - retroactive period
     L - lag period
     ret - 收盘涨跌幅 or 结算涨跌幅
     limit - 成交量阈值，剔除低于该值的品种
    
    """

    def __init__(self,R,L,ret,limit):

        self.R = R
        self.L = L
        self.ret = ret
        self.limit = limit
        
    def ts_mom(self):      #时序动量
        
        table = df.copy()
        
        table['ts_mom'] = 0
        for j in PZ_list:
            table.loc[(table['品种']==j), 'ts_mom'] = table.loc[table['品种']==j][self.ret].shift(self.L+1).rolling(self.R).apply(multisum)
        table = table.dropna()
        
        table['ts_position'] = np.sign(table['ts_mom'])
        
        #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #多空相减
        long_short = LongShort(table,self.ret,'ts_position')
#         table['factor_return'] = long_short['factor_return']
        
        return table,long_short

In [11]:
class CrossSection_ls(object): 
    """截面类因子，品种收益多头减空头
    
    Parameters:
     R - retroactive period
     L - lag period
     ret - 收盘涨跌幅 or 结算涨跌幅
     limit - 成交量阈值，剔除低于该值的品种
    
    """

    def __init__(self,R,L,ret,limit):

        self.R = R
        self.L = L
        self.ret = ret
        self.limit = limit
            
    def cs_mom(self):     
        """计算截面动量因子
        
        """
        ts_factor = TimeSeries_ls(self.R,self.L,self.ret,self.limit)
        ts_table,ts_mom = ts_factor.ts_mom()
        table = ts_table.copy()
        
        #时序动量在截面上排序
        table['rank'] = table.groupby(by='时间')['ts_mom'].rank(axis=0, method='dense', ascending=False)
        
        table = cs_position(table)
        long_short =  LongShort(table,ret,'cs_position')
        return table,long_short
    
    def skew(self):      
        """计算偏度因子
        
        """
        table = df.copy()
        
        table['skew'] = 0
        for j in PZ_list:
            factor = table.loc[table['品种']==j][self.ret].shift(self.L+1).rolling(self.R).apply(lambda x:x.skew())     #无偏估计
        #     factor = table.loc[table['品种']==j]['lnret'].shift(L+1).rolling(R).apply(lambda x:stats.skew(x))   #有偏估计
            table.loc[(table['品种']==j), 'skew'] = factor
        table = table.dropna()
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #持仓变动因子在截面上排序
        table['rank'] = table.groupby(by='时间')['skew'].rank(axis=0, method='dense', ascending=True)
        table = cs_position(table)
        long_short =  LongShort(table,ret,'cs_position')
        
        return table,long_short
    
    def cv(self): 
        """计算波动因子
        
        """
        table = df.copy()
        
        table['variance'] = 0
        table['avg_ret'] = 0
        table['cv'] = 0
        for j in PZ_list:
            table.loc[(table['品种']==j), 'variance'] =  table.loc[table['品种']==j][self.ret].shift(self.L+1).rolling(self.R).apply(lambda x:np.var(x))
            table.loc[(table['品种']==j), 'avg_ret'] =  table.loc[table['品种']==j][self.ret].shift(self.L+1).rolling(self.R).apply(lambda x:np.mean(x))
            table.loc[(table['品种']==j), 'cv'] = table.loc[(table['品种']==j), 'variance']/abs(table.loc[(table['品种']==j), 'avg_ret'])
        table = table.dropna() 
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #变异系数在截面上排序
        table['rank'] = table.groupby(by='时间')['cv'].rank(axis=0, method='dense', ascending=False)
        table = cs_position(table)
        long_short =  LongShort(table,ret,'cs_position')
        
        return table,long_short
    
    def liquid(self):
        """计算流动性因子
        
        """
        table = df.copy()
        
        #计算流动性因子
        table['Amivest'] = abs(table[self.ret])/table['成交金额'] 
        for j in PZ_list:
            factor = table.loc[table['品种']==j]['Amivest'].rolling(self.R).mean().shift(self.L+1)     
            table.loc[(table['品种']==j), 'liquid'] = factor
        table = table.dropna() 
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #流动性因子在截面上排序
        table['rank'] = table.groupby(by='时间')['liquid'].rank(axis=0, method='dense', ascending=False)
        table = cs_position(table)
        long_short =  LongShort(table,ret,'cs_position')
        
        return table,long_short
        
    def ma(self):
        """计算均价突破因子
        
        """
        table = df.copy()

        #计算均价MA
        for j in PZ_list:
            table.loc[(table['品种']==j), 'price_ave'] = table.loc[table['品种']==j][self.ret[:2]+'价'].rolling(self.R).mean()
            table.loc[(table['品种']==j), 'price_t-20'] = table.loc[table['品种']==j][self.ret[:2]+'价'].shift(self.R-1) 
            table.loc[(table['品种']==j), 'sigma'] = table.loc[table['品种']==j][self.ret].rolling(self.R).std() 

        #均价突破因子
        table['ma'] = ((table[self.ret[:2]+'价']-table['price_ave'])/table['sigma']/table['price_t-20']).shift(1)
        table = table.dropna()
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #均价突破因子在截面上排序
        table['rank'] = table.groupby(by='时间')['ma'].rank(axis=0, method='dense', ascending=False)
        table = cs_position(table)
        long_short =  LongShort(table,ret,'cs_position')
        
        return table,long_short

    def delta_oi(self):
        """计算持仓变化因子
        
        """
        table = df.copy()
        table = pd.merge(table,multi,on='品种',how='left')
        
        #计算持仓变化因子
        table['持仓金额'] = table['持仓量']*table['合约乘数']*table['结算价']
        for j in PZ_list:
            table.loc[(table['品种']==j), 'delta_oi'] = (table.loc[table['品种']==j]['持仓金额'] - table.loc[table['品种']==j]['持仓金额'].shift(self.R)).shift(1)
        table = table.dropna()
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #持仓变动因子在截面上排序
        table['rank'] = table.groupby(by='时间')['delta_oi'].rank(axis=0, method='dense', ascending=False)
        table = cs_position(table)
        long_short =  LongShort(table,self.ret,'cs_position')
        
        return table,long_short
    
    def beta(self):
        """计算beta因子
        
        """
        table = df.copy()

        table['year'] = table['时间'].dt.year
        table['month'] = table['时间'].dt.month
        
        #合并beta因子
        betas = pd.read_excel('./A.7 - Beta因子/beta系数1.xlsx')
        betas['lag时间'] = 0
        for z in range(len(betas)):
            betas.loc[z,'lag时间'] = betas.loc[z,'时间'] + relativedelta(months = +1)
        betas['lag时间'] = pd.to_datetime(betas['lag时间'])
        betas['year'] = betas['lag时间'].dt.year
        betas['month'] = betas['lag时间'].dt.month
        del betas['时间']
        del betas['lag时间']
        table = pd.merge(table,betas,how='left',on=['品种','year','month'])
        table['beta'].fillna(method = 'ffill',inplace=True)
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #beta因子在截面上排序
        table['rank'] = table.groupby(by='时间')['beta'].rank(axis=0, method='dense', ascending=False)
        table = cs_position(table)
        long_short =  LongShort(table,ret,'cs_position')
        
        return table,long_short

In [12]:
def NetValues(df):
    """计算因子净值序列
    
    Parameters:
     df - 包含因子return的dataframe
     
    Returns:
     包含因子虚拟净值序列的dataframe
    """
    
    net_values = pd.DataFrame(df.groupby(by='时间')['factor_return'].mean())
    net_values['1+ret'] = 1 + net_values['factor_return']
    net_values['net_value'] = net_values['1+ret'].cumprod()
    
    return net_values

计算因子值

In [39]:
single_factors = ['skew', 'liquid', 'ma', 'delta_oi','beta']
ave_R = [5,20,60,120]
single_R = [250,250,20,22,0]
L = 0
limit = 0
ret = '收盘涨跌幅'
factors = pd.DataFrame()

# #计算时序动量因子值
for r in ave_R:
    ts_factor = TimeSeries_ls(r,L,ret,limit)
    table,ts_mom = ts_factor.ts_mom()
    factor_value = table[['时间','品种','ts_mom']]
    factor_value.rename(columns={'ts_mom':'ts_mom'+'_'+str(r)},inplace=True)
    try:
        factors = pd.merge(factors,factor_value,on=['品种','时间'],how='outer')
    except:
        factors = factor_value
        
# #计算截面类因子值
# 1 多周期因子 cv
for r in ave_R:
    cs_factor = CrossSection_ls(r,L,ret,limit)
    table,factor = getattr(cs_factor,'cv')()
    factor_value = table[['时间','品种','cv']]
    factor_value.rename(columns={'cv':'cv'+'_'+str(r)},inplace=True)
    factors = pd.merge(factors,factor_value,on=['品种','时间'],how='outer')
    
# 2 单周期因子
for i,fac in enumerate(single_factors):
    cs_factor = CrossSection_ls(single_R[i],L,ret,limit)
    table,factor = getattr(cs_factor,fac)()
    factor_value = table[['时间','品种',fac]]
    factor_value.rename(columns={fac:fac+'_'+str(single_R[i])},inplace=True)
    factors = pd.merge(factors,factor_value,on=['品种','时间'],how='outer')
    

In [40]:
factors

,时间,品种,ts_mom_5,ts_mom_20,ts_mom_60,ts_mom_120,cv_5,cv_20,cv_60,cv_120,skew_250,liquid_250,ma_20,delta_oi_22,beta_0
0,2014-01-29,A,-0.003580,0.041005,NaN,NaN,0.017808,0.010393,NaN,NaN,NaN,NaN,5.239465,NaN,-1.242460
1,2014-01-30,A,0.001722,0.033565,NaN,NaN,0.017913,0.012157,NaN,NaN,NaN,NaN,4.723029,NaN,-1.242460
2,2014-02-07,A,-0.002119,0.032908,NaN,NaN,0.008991,0.012470,NaN,NaN,NaN,NaN,3.688172,NaN,-1.191035
3,2014-02-10,A,0.002783,0.036459,NaN,NaN,0.013752,0.011444,NaN,NaN,NaN,NaN,3.734646,917376600.0,-1.191035
4,2014-02-11,A,0.001926,0.036520,NaN,NaN,0.018206,0.011425,NaN,NaN,NaN,NaN,3.222041,669258080.0,-1.191035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98423,2023-01-19,sc,0.081539,0.046770,-0.182582,0.016909,0.004012,0.157226,0.138134,3.973367,-0.024861,2.430502e-13,1.819824,-4351634200.000002,-0.887816
98424,2023-01-20,sc,0.035177,0.009250,-0.198274,-0.069110,0.045386,0.858593,0.129446,0.927284,-0.022668,2.435465e-13,0.435502,-4780572899.999998,-0.887816
98425,2023-01-30,sc,0.048192,0.028269,-0.189993,-0.015117,0.039468,0.302646,0.137326,4.238710,-0.036340,2.431377e-13,1.609231,-5827610700.0,-0.887816
98426,2023-01-31,sc,0.019806,0.011535,-0.218783,-0.082185,0.133299,0.779452,0.119937,0.757568,-0.021415,2.468492e-13,0.549758,-6132538600.0,-0.887816


In [41]:
factors_ori = pd.read_excel('CTA历史因子值.xlsx')
factors_ori

,时间,品种,ts_mom_5,ts_mom_20,ts_mom_60,ts_mom_120,cv_5,cv_20,cv_60,cv_120,skew_250,liquid_250,ma_20,delta_oi_22,beta_0,last_update
0,2014-01-29,A,-0.003580,0.041005,NaN,NaN,0.017808,0.010393,NaN,NaN,NaN,NaN,5.239465,NaN,-1.242460,-
1,2014-01-30,A,0.001722,0.033565,NaN,NaN,0.017913,0.012157,NaN,NaN,NaN,NaN,4.723029,NaN,-1.242460,-
2,2014-02-07,A,-0.002119,0.032908,NaN,NaN,0.008991,0.012470,NaN,NaN,NaN,NaN,3.688172,NaN,-1.191035,-
3,2014-02-10,A,0.002783,0.036459,NaN,NaN,0.013752,0.011444,NaN,NaN,NaN,NaN,3.734646,9.173766e+08,-1.191035,-
4,2014-02-11,A,0.001926,0.036520,NaN,NaN,0.018206,0.011425,NaN,NaN,NaN,NaN,3.222041,6.692581e+08,-1.191035,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98376,2023-01-18,sc,0.047534,0.021294,-0.201825,-0.043453,0.007326,0.309060,0.120543,1.547478,-0.024195,2.434424e-13,0.461891,-3.799926e+09,-0.887816,-
98377,2023-01-19,sc,0.081539,0.046770,-0.182582,0.016909,0.004012,0.157226,0.138134,3.973367,-0.024861,2.430502e-13,1.819824,-4.351634e+09,-0.887816,-
98378,2023-01-20,sc,0.035177,0.009250,-0.198274,-0.069110,0.045386,0.858593,0.129446,0.927284,-0.022668,2.435465e-13,0.435502,-4.780573e+09,-0.887816,-
98379,2023-01-30,sc,0.048192,0.028269,-0.189993,-0.015117,0.039468,0.302646,0.137326,4.238710,-0.036340,2.431377e-13,1.609231,-5.827611e+09,-0.887816,-


In [42]:
insert_date = factors['时间'][len(factors)-1]
add = factors.loc[factors['时间']==insert_date]
add

,时间,品种,ts_mom_5,ts_mom_20,ts_mom_60,ts_mom_120,cv_5,cv_20,cv_60,cv_120,skew_250,liquid_250,ma_20,delta_oi_22,beta_0
2189,2023-02-01,A,0.014641,0.065383,0.037150,-0.019338,0.003574,0.011597,0.081964,0.418306,0.491908,1.103037e-12,6.527434,-2223007360.0,-1.183467
4379,2023-02-01,AG,-0.023099,-0.030914,0.132656,0.153048,0.074222,0.140404,0.102095,0.166751,-0.004708,2.930650e-13,-1.082644,10836221760.0,1.521104
6569,2023-02-01,AL,0.017252,0.014909,0.083406,0.033599,0.082075,0.251837,0.086615,0.576917,-0.629316,4.327086e-13,1.413059,8510281975.0,-1.644070
7662,2023-02-01,AP,0.015024,0.100575,0.149708,0.081532,0.020686,0.052445,0.086912,0.347105,0.165952,8.842813e-13,2.072289,2020341400.0,6.079089
9852,2023-02-01,AU,0.001223,0.028036,0.067369,0.082390,0.127263,0.014509,0.027026,0.039909,-0.104052,1.320444e-13,2.520614,14270767340.0,8.177492
12042,2023-02-01,B,0.018129,0.022032,0.049108,0.150869,0.030248,0.102301,0.169171,0.138963,0.281619,5.529830e-12,-0.371760,642823370.0,0.000476
14232,2023-02-01,BU,0.036875,0.048634,0.080669,0.017117,0.017871,0.062190,0.195599,1.779676,0.018932,9.044576e-13,2.108294,8779201720.0,0.357236
16422,2023-02-01,C,-0.014393,0.023981,-0.011534,0.058867,0.035441,0.039127,0.246200,0.108613,-0.428161,5.686117e-13,-1.051429,-5584691970.0,5.584521
18612,2023-02-01,CF,0.049599,0.074487,0.208110,0.063085,0.006199,0.013462,0.029422,0.405524,-0.456868,3.153393e-13,6.178266,6339854500.0,1.035858
19378,2023-02-01,CJ,0.043773,0.064789,0.006062,-0.067546,0.014060,0.031328,1.648324,0.257266,0.317291,1.378469e-11,4.640008,1052664800.0,-1.674244


In [44]:
factors = factors_ori.append(add)
factors.reset_index(drop=True)
factors = factors.sort_values(by=['品种','时间'])
factors

,时间,品种,ts_mom_5,ts_mom_20,ts_mom_60,ts_mom_120,cv_5,cv_20,cv_60,cv_120,skew_250,liquid_250,ma_20,delta_oi_22,beta_0,last_update
0,2014-01-29,A,-0.003580,0.041005,NaN,NaN,0.017808,0.010393,NaN,NaN,NaN,NaN,5.239465,NaN,-1.242460,-
1,2014-01-30,A,0.001722,0.033565,NaN,NaN,0.017913,0.012157,NaN,NaN,NaN,NaN,4.723029,NaN,-1.242460,-
2,2014-02-07,A,-0.002119,0.032908,NaN,NaN,0.008991,0.012470,NaN,NaN,NaN,NaN,3.688172,NaN,-1.191035,-
3,2014-02-10,A,0.002783,0.036459,NaN,NaN,0.013752,0.011444,NaN,NaN,NaN,NaN,3.734646,917376600.0,-1.191035,-
4,2014-02-11,A,0.001926,0.036520,NaN,NaN,0.018206,0.011425,NaN,NaN,NaN,NaN,3.222041,669258080.0,-1.191035,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98377,2023-01-19,sc,0.081539,0.046770,-0.182582,0.016909,0.004012,0.157226,0.138134,3.973367,-0.024861,2.430502e-13,1.819824,-4351634200.000002,-0.887816,-
98378,2023-01-20,sc,0.035177,0.009250,-0.198274,-0.069110,0.045386,0.858593,0.129446,0.927284,-0.022668,2.435465e-13,0.435502,-4780572899.999998,-0.887816,-
98379,2023-01-30,sc,0.048192,0.028269,-0.189993,-0.015117,0.039468,0.302646,0.137326,4.238710,-0.036340,2.431377e-13,1.609231,-5827610700.0,-0.887816,-
98380,2023-01-31,sc,0.019806,0.011535,-0.218783,-0.082185,0.133299,0.779452,0.119937,0.757568,-0.021415,2.468492e-13,0.549758,-6132538600.0,-0.887816,NaN


In [15]:
#每月更新CPI后执行，注意修改当日日期
# factors['last_update'] = '-'
# factors.loc[(factors['时间']>='2023-01-01')&(factors['时间']<'2023-01-18'),'last_update'] = '2023-01-18'
# factors

,时间,品种,ts_mom_5,ts_mom_20,ts_mom_60,ts_mom_120,cv_5,cv_20,cv_60,cv_120,skew_250,liquid_250,ma_20,delta_oi_22,beta_0,last_update
0,2014-01-29,A,-0.003580,0.041005,NaN,NaN,0.017808,0.010393,NaN,NaN,NaN,NaN,5.239465,NaN,-1.242460,-
1,2014-01-30,A,0.001722,0.033565,NaN,NaN,0.017913,0.012157,NaN,NaN,NaN,NaN,4.723029,NaN,-1.242460,-
2,2014-02-07,A,-0.002119,0.032908,NaN,NaN,0.008991,0.012470,NaN,NaN,NaN,NaN,3.688172,NaN,-1.191035,-
3,2014-02-10,A,0.002783,0.036459,NaN,NaN,0.013752,0.011444,NaN,NaN,NaN,NaN,3.734646,917376600.0,-1.191035,-
4,2014-02-11,A,0.001926,0.036520,NaN,NaN,0.018206,0.011425,NaN,NaN,NaN,NaN,3.222041,669258080.0,-1.191035,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98329,2023-01-17,sc,0.030333,0.007038,-0.207732,-0.052370,0.019608,0.929439,0.116565,1.282237,-0.018995,2.429566e-13,0.029801,-3961504799.999996,-0.887816,2023-01-18
98330,2023-01-18,sc,0.047534,0.021294,-0.201825,-0.043453,0.007326,0.309060,0.120543,1.547478,-0.024195,2.434424e-13,0.461891,-3799926200.0,-0.887816,-
98331,2023-01-19,sc,0.081539,0.046770,-0.182582,0.016909,0.004012,0.157226,0.138134,3.973367,-0.024861,2.430502e-13,1.819824,-4351634200.000002,-0.887816,-
98332,2023-01-20,sc,0.035177,0.009250,-0.198274,-0.069110,0.045386,0.858593,0.129446,0.927284,-0.022668,2.435465e-13,0.435502,-4780572899.999998,-0.887816,-


In [45]:
factors.to_excel('CTA历史因子值.xlsx', index=False)

计算因子组合回报

生成【原始】的因子组合收益

In [46]:
cs_ave_factors = ['cv','cs_mom']
single_factors = ['skew', 'liquid', 'ma', 'delta_oi','beta']
ave_R = [5,20,60,120]
single_R = [250,250,20,22,0]
L = 0
limit = 0
ret = '收盘涨跌幅'
fac_dict = {}
ave_factor = pd.DataFrame(columns=['fac20','fac60','fac120'])

# #计算时序动量组合回报
for r in ave_R:
        ts_factor = TimeSeries_ls(r,L,ret,limit)
        table,ts_mom = ts_factor.ts_mom()
        a = NetValues(ts_mom)
        fac_dict['ts_mom_short'+'_'+str(r)] = a['factor_return']
        a.to_excel('.//A.1 - 动量因子//时序动量因子_R=%s收盘多空头.xlsx'%r)

# #计算截面类因子组合回报
# 1 需要平均
for fac in cs_ave_factors:
    for r in ave_R:
            cs_factor = CrossSection_ls(r,L,ret,limit)
            table,factor = getattr(cs_factor,fac)()
            b = NetValues(factor)
            fac_dict[fac+'_'+str(r)] = b['factor_return']
            if fac == 'cs_mom':
                b.to_excel('.//A.1 - 动量因子//截面动量因子_R=%s收盘多空头.xlsx'%r)
            if fac == 'cv':
                b.to_excel('.//A.3 - 波动因子//波动因子_R=%s收盘多空头.xlsx'%r)

# 2 不需要平均
for i,fac in enumerate(single_factors):
    cs_factor = CrossSection_ls(single_R[i],L,ret,limit)
    table,factor = getattr(cs_factor,fac)()
    c = NetValues(factor)
    fac_dict[fac] = c['factor_return']
    if fac == 'skew':
        c.to_excel('.//A.2 - 偏度因子//偏度因子_R=%s收盘多空头.xlsx'%single_R[i])
    if fac == 'liquid':
        c.to_excel('.//A.4 - 流动性因子//流动性因子_R=%s收盘多空头成交金额.xlsx'%single_R[i])
    if fac == 'ma':
        c.to_excel('.//A.5 - 均价突破因子//均价突破因子_R=%s收盘多空头.xlsx'%single_R[i])
    if fac == 'delta_oi':
        c.to_excel('.//A.6 - 持仓变化因子//持仓变化因子_R=%s收盘多空头.xlsx'%single_R[i])
    if fac == 'beta':
        c.to_excel('./A.7 - Beta因子/beta因子_36个月.xlsx')

In [47]:
factor_returns = pd.DataFrame(fac_dict)
factor_returns

,ts_mom_short_5,ts_mom_short_20,ts_mom_short_60,ts_mom_short_120,cv_5,cv_20,cv_60,cv_120,cs_mom_5,cs_mom_20,cs_mom_60,cs_mom_120,skew,liquid,ma,delta_oi,beta
时间,,,,,,,,,,,,,,,,,
2014-01-29,0.005356,0.000044,NaN,NaN,-0.001009,-0.004773,NaN,NaN,0.007709,-0.001177,NaN,NaN,NaN,NaN,0.003038,NaN,0.005084
2014-01-30,0.001035,0.003932,NaN,NaN,0.001241,0.000824,NaN,NaN,0.006241,0.004023,NaN,NaN,NaN,NaN,0.004880,NaN,-0.003435
2014-02-07,0.001870,-0.004008,NaN,NaN,-0.006057,0.008138,NaN,NaN,-0.015633,-0.006893,NaN,NaN,NaN,NaN,-0.008273,NaN,-0.006435
2014-02-10,-0.001937,-0.004768,NaN,NaN,0.008675,0.000656,NaN,NaN,-0.003658,-0.012855,NaN,NaN,NaN,NaN,-0.006823,-0.006090,0.006869
2014-02-11,0.003290,0.003015,NaN,NaN,-0.000326,-0.004335,NaN,NaN,0.002659,0.004161,NaN,NaN,NaN,NaN,0.005734,0.002495,-0.005007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-19,-0.012108,-0.009069,0.005811,0.009438,0.006087,0.004260,-0.003752,0.003843,-0.009590,-0.003307,0.008163,0.011827,0.008721,0.004317,-0.005722,-0.002408,-0.003442
2023-01-20,0.005055,0.003565,-0.001924,-0.002113,-0.002057,0.011537,-0.008149,0.006519,0.018592,0.002792,0.006101,0.005232,-0.001206,-0.013196,0.005998,-0.005790,0.002304
2023-01-30,-0.012139,-0.007364,-0.009446,0.002472,-0.002711,0.003055,0.003214,-0.002797,-0.018463,-0.015989,-0.011840,-0.005089,-0.005469,0.000318,-0.017527,-0.011756,0.005577


In [48]:
factor_returns.to_excel('CTA历史因子组合收益.xlsx')

生成【平均后】的因子组合收益(平均还有点问题！！！)

In [ ]:
'''
cs_ave_factors = ['cv','cs_mom']
single_factors = ['skew', 'liquid', 'ma', 'delta_oi','beta']
ave_R = [5,20,60,120]
single_R = [250,250,20,22,0]
L = 0
limit = 0
ret = '收盘涨跌幅'
fac_dict = {}
ave_factor = pd.DataFrame(columns=['fac20','fac60','fac120'])

# #计算时序动量组合回报
for r in ave_R:
    if r == 5:
        ts_factor = TimeSeries_ls(r,L,ret,limit)
        table,ts_mom = ts_factor.ts_mom()
        a = NetValues(ts_mom)
        fac_dict['ts_mom_short'] = a['factor_return']
#         a.to_excel('.//A.1 - 动量因子//时序动量因子_R=%s收盘多空头.xlsx'%r)
    else:
        ts_factor = TimeSeries_ls(r,L,ret,limit)
        table,ts_mom = ts_factor.ts_mom()
        a = NetValues(ts_mom)
        ave_factor['fac%s'%r] = a['factor_return']
#         a.to_excel('.//A.1 - 动量因子//时序动量因子_R=%s收盘多空头.xlsx'%r)
ave_factor['ts_mom_long'] = ave_factor.iloc[:,:3].mean(axis=1)
fac_dict['ts_mom_long'] = ave_factor['ts_mom_long'] 
        

# #计算截面类因子组合回报
# 1 需要平均
for fac in cs_ave_factors:
    for r in ave_R:
        if r == 5:
            cs_factor = CrossSection_ls(r,L,ret,limit)
            table,factor = getattr(cs_factor,fac)()
            b = NetValues(factor)
            fac_dict[fac+'_short'] = b['factor_return']
#             if fac == 'cs_mom':
#                 b.to_excel('.//A.1 - 动量因子//截面动量因子_R=%s收盘多空头.xlsx'%r)
#             if fac == 'cv':
#                 b.to_excel('.//A.3 - 波动因子//波动因子_R=%s收盘多空头.xlsx'%r)
        else:
            cs_factor = CrossSection_ls(r,L,ret,limit)
            table,factor = getattr(cs_factor,fac)()
            b = NetValues(factor)
            ave_factor['fac%s'%r] = b['factor_return']
#             if fac == 'cs_mom':
#                 b.to_excel('.//A.1 - 动量因子//截面动量因子_R=%s收盘多空头.xlsx'%r)
#             if fac == 'cv':
#                 b.to_excel('.//A.3 - 波动因子//波动因子_R=%s收盘多空头.xlsx'%r)
    ave_factor['cs_mom_long'] = ave_factor.iloc[:,:3].mean(axis=1)
    fac_dict[fac+'_long'] = ave_factor['cs_mom_long']
    
# 2 不需要平均
for i,fac in enumerate(single_factors):
    cs_factor = CrossSection_ls(single_R[i],L,ret,limit)
    table,factor = getattr(cs_factor,fac)()
    c = NetValues(factor)
    fac_dict[fac] = c['factor_return']
#     if fac == 'skew':
#         c.to_excel('.//A.2 - 偏度因子//偏度因子_R=%s收盘多空头.xlsx'%single_R[i])
#     if fac == 'liquid':
#         c.to_excel('.//A.4 - 流动性因子//流动性因子_R=%s收盘多空头成交金额.xlsx'%single_R[i])
#     if fac == 'ma':
#         c.to_excel('.//A.5 - 均价突破因子//均价突破因子_R=%s收盘多空头.xlsx'%single_R[i])
#     if fac == 'delta_oi':
#         c.to_excel('.//A.6 - 持仓变化因子//持仓变化因子_R=%s收盘多空头.xlsx'%single_R[i])
'''

In [ ]:
# factor_returns1 = pd.DataFrame(fac_dict)
# factor_returns1